In [ ]:
%load_ext autoreload
%autoreload 2
from oxeo.water.datamodules.samplers import RandomSampler, GridSampler
from oxeo.water.datamodules.datasets import TileDataset, UnionDataset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
from skimage import img_as_float
from skimage.exposure import equalize_adapthist, rescale_intensity
import numpy as np
from oxeo.water.models.utils import load_tile, TilePath, tile_from_id

In [ ]:
from torchvision.transforms.functional import resized_crop

class SelectBands:
    """Computes sample virtual arrays to numpy array"""

    def __init__(self, bands):
        self.bands = bands

    def __call__(self, sample):
        sample["image"] = sample["image"][self.bands]
        return sample
    

In [ ]:
from torchvision.transforms import Resize, Compose

In [ ]:
TilePath(tile_from_id("43_P_10000_65_137"), "landsat-5").tile.id

In [ ]:
datasets = []
tiles = []
for t in ["43_P_10000_65_137","43_P_10000_65_138"
          ]:
    
    tiles.append(TilePath(tile_from_id(t), "landsat-5"))
ds = TileDataset(tiles, transform=SelectBands(bands=[3,2,1]), masks=("pekel",), target_size=1000)


    

In [ ]:
for c in ["landsat-7", "landsat-8"]:
    tiles = []
    for t in [ "43_P_10000_65_138", "43_P_10000_66_137","43_P_10000_66_138","43_P_10000_67_137"]:
        tiles.append(TilePath(tile_from_id(t), c))

    ds2 = TileDataset(tiles, transform=SelectBands(bands=[3,2,1]), masks=("pekel",), target_size=1000)

    ds = UnionDataset(ds, ds2)
          

In [ ]:
len(ds)

In [ ]:

sampler = RandomSampler(ds, 1000, 1000, 200)
sampler = GridSampler(ds, 1000, 1000)#


In [ ]:
def worker_init_fn(worker_id):
    """Configures each dataset worker process.

    Just has one job!  To call SatelliteDataset.per_worker_init().
    """
    # get_worker_info() returns information specific to each worker process.
    worker_info = torch.utils.data.get_worker_info()
    if worker_info is None:
        print("worker_info is None!")
    else:
        dataset_obj = worker_info.dataset  # The Dataset copy in this worker process.
        dataset_obj.per_worker_init()


dataloader = DataLoader(ds, batch_size=1, 
                        sampler=sampler, 
                        num_workers=3, 
                        worker_init_fn=worker_init_fn)

len(dataloader)

In [ ]:
for sample in dataloader:
    img = img_as_float(sample["image"][0].numpy().transpose(1,2,0))
    mask = sample["pekel"].numpy()[0].squeeze()
    vmin, vmax = np.percentile(img, q=(0, 98))
    img = rescale_intensity(img,in_range=(vmin,vmax))
    plt.imshow(img)
    plt.show()
    plt.imshow(mask)
    plt.show()